## Language Model pt-BR: Preparação de dados
Fred Guth 24/9/18

Baseado em código de https://github.com/monilouise/deep-learning

In [1]:
from pathlib import Path
import json
import re
import pandas as pd
import numpy as np
import html
from fastai.text import *

In [4]:
# set path for language model files
LM_PATH=Path('./lm/')
CORPORA_PATH = Path('./corpora/')
DATA_PATH = Path('./data/text')

### Original Data: Wikipedia PT-BR

Todos artigos da Wikipedia em pt-br em 01/09/2018 na versão corrente. Não traz versões antigas, nem comentários e outros metadados.

In [ ]:
 !wget https://dumps.wikimedia.org/ptwiki/20180901/ptwiki-20180901-pages-meta-current.xml.bz2 --directory-prefix=data

In [ ]:
!wget https://raw.githubusercontent.com/attardi/wikiextractor/master/WikiExtractor.py --directory-prefix=data

In [ ]:
!python ./data/WikiExtractor.py -o data/text/ ./data/ptwiki-latest-pages-articles-multistream.xml -s --json

WikiExtractor converte xml com todas as páginas da wikipedia em português em arquivos json organizados em diferentes folders.

Agora partirmos para geração de um corpus em português

In [5]:
LANG_FILENAMES = [str(f) for f in DATA_PATH.rglob("*/*")]

In [6]:
LANG_TEXT = []
for i in LANG_FILENAMES:
    for line in open(i):
        LANG_TEXT.append(json.loads(line))
        
LANG_TEXT = pd.DataFrame(LANG_TEXT)

Após colocar todos textos em um dataframe, ordenamos em ordem decrescente de tamanho. 

In [7]:
s = LANG_TEXT.text.str.len().sort_values(ascending=False).index
LANG_TEXT = LANG_TEXT.reindex(s)
LANG_TEXT = LANG_TEXT.reset_index(drop=True)

# Selecionamos os maiores textos (~1.0%)

In [15]:
LANG_TEXT.shape[0]//100

10042

In [16]:
LANG_TEXT = LANG_TEXT[0:LANG_TEXT.shape[0]//100]

In [17]:
LANG_TEXT.to_csv(f"{CORPORA_PATH}/Wiki_PT_Corpus.csv", index=False)

In [18]:
LANG_TEXT  = pd.read_csv(f"{CORPORA_PATH}/Wiki_PT_Corpus.csv")

In [19]:
(LANG_TEXT.assign(labels = 0)
    .pipe(lambda x: x[['labels', 'text']])
    .to_csv(f"{CORPORA_PATH}/Wiki_PT_Corpus2.csv", header=None, index=False))

In [20]:
LANG_TEXT[:3]

,id,text,title,url
0,2210422,Armadas da Índia\n\nAs Armadas da Índia foram ...,Armadas da Índia,https://pt.wikipedia.org/wiki?curid=2210422
1,1018503,Lista de vilões de Ben 10\n\nNa lista a seguir...,Lista de vilões de Ben 10,https://pt.wikipedia.org/wiki?curid=1018503
2,2737450,História da televisão no Brasil\n\nA história ...,História da televisão no Brasil,https://pt.wikipedia.org/wiki?curid=2737450


In [21]:
# Getting rid of the title name in the text field
def split_title_from_text(text):
    words = text.split("\n\n")
    if len(words) >= 2:
        return ''.join(words[1:])
    else:
        return ''.join(words)
    

In [22]:
LANG_TEXT['text'].apply(lambda x: len(x.split(" "))).sum()

46411972

In [23]:
len(set(''.join(LANG_TEXT['text'].values).split(" ")))

2042620

46.411.972 palavras e 2.042.620 palavras únicas

## Tokenização

In [24]:
LANG_TEXT = pd.read_csv(f"{CORPORA_PATH}/Wiki_PT_Corpus2.csv", header=None)#, chunksize=5000)

In [25]:
BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

In [26]:
re1 = re.compile(r'  +')

def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

In [27]:
def get_texts(df, n_lbls=1):
    
    labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
    texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
    for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls} ' + df[i].astype(str)

    texts = texts.apply(fixup).values.astype(str)
    tok = Tokenizer.proc_all_mp(partition_by_cores(texts), lang='pt')

    return tok, list(labels)



In [28]:
def get_all(df, n_lbls):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls)
        tok += tok_;
        labels += labels_
    return tok, labels

In [29]:
LANG_TEXT.shape

(9942, 2)

In [30]:
tok_total, trn_labels = get_texts(LANG_TEXT, 1)

In [31]:
freq = Counter(p for o in tok_total for p in o)

In [32]:
len(freq)

556634

O dicionário tem 556 mil tokens

The *vocab* is the **unique set of all tokens** in our dataset. The vocab provides us a way for us to simply replace each word in our datasets with a unique integer called an index.

In a large corpus of data one might find some rare words which are only used a few times in the whole dataset. We discard such rare words and avoid trying to learn meaningful patterns out of them.

Here we have set a minimum frequency of occurence to 2 times. It has been observed by NLP practicioners that a maximum vocab of 60k usually yields good results for classification tasks. So we set maz_vocab to 60000.

In [33]:
max_vocab = 100000
min_freq = 2

In [34]:
itos = [o for o,c in freq.most_common(max_vocab) if c>min_freq]
itos.insert(0, '_pad_')
itos.insert(0, '_unk_')

We create a reverse mapping called stoi which is useful to lookup the index of a given token. stoi also has the same number of elements as itos. We use a high performance container called [collections.defaultdict](https://docs.python.org/2/library/collections.html#collections.defaultdict) to store our stoi mapping.

In [35]:
stoi = collections.defaultdict(lambda:0, {v:k for k,v in enumerate(itos)})
len(itos)

100002

297 (8603-8306) palavras a mais após filtro de mínimo de ocorrências! (Borela)

In [36]:
trn_lm = np.array([[stoi[o] for o in p] for p in tok_total])
# val_lm = np.array([[stoi[o] for o in p] for p in tok_val])

In [37]:
np.save(LM_PATH/'tmp'/'trn_ids.npy', trn_lm)
# np.save(LM_PATH/'tmp'/'val_ids.npy', val_lm)
pickle.dump(itos, open(LM_PATH/'tmp'/'itos.pkl', 'wb'))

In [ ]:
trn_lm = np.load(LM_PATH/'tmp'/'trn_ids.npy')
# val_lm = np.load(LM_PATH/'tmp'/'val_ids.npy')
itos = pickle.load(open(LM_PATH/'tmp'/'itos.pkl', 'rb'))

Passo para rexecução 

In [38]:
vs=len(itos)
vs,len(trn_lm)

(100002, 9942)

In [41]:
tok_trn = tok_total

In [42]:
' '.join(tok_trn[0]+tok_trn[1]+tok_trn[2]+tok_trn[3]+tok_trn[4])

'\n xbos xfld 1 armadas da índia \n\n as armadas da índia foram as frotas de navios enviadas anualmente por portugal para a índia , principalmente para goa . estas armadas faziam a chamada carreira da índia , seguindo a rota do cabo iniciada por vasco da gama em 1497 - 1499 com a descoberta do caminho marítimo para a índia . contudo este artigo , compreende aqui as armadas anteriores , enviadas pelo infante d. henrique a partir de 1412 , e que contribuíram para a descoberta d esta rota . \n\n desde a sua descoberta , a rota do cabo foi dominada pelos portugueses , tendo sido percorrida , de 1498 a 1635 , por 917 partidas de armadas do tejo para uma viagem que demorava cerca de seis meses a chegar a o destino . durante mais de oitenta anos , as armadas da índia puderam circular pela rota do cabo . o seu tamanho aumentou progressivamente desde os 120 tonéis da “ s. gabriel ” . as naus típicas do tempo de d. manuel i deslocavam 400 toneladas e atingiram as 900 toneladas durante o reinado 

In [43]:
' '.join(str(o) for o in trn_lm[0:4])

'[31, 446, 447, 110, 2244, 12, 1194, 14, 29, 2244, 12, 1194, 51, 29, 14005, 3, 978, 7583, 3352, 23, 354, 16, 5, 1194, 2, 321, 16, 10014, 4, 310, 2244, 2966, 5, 461, 469, 12, 1194, 2, 1738, 5, 2929, 11, 1291, 3485, 23, 2757, 12, 2426, 9, 33066, 13, 30320, 18, 5, 1649, 11, 706, 6191, 16, 5, 1194, 4, 570, 62, 1081, 2, 5718, 1285, 29, 2244, 945, 2, 7583, 48, 6015, 541, 1095, 5, 148, 3, 55631, 2, 7, 10, 5201, 16, 5, 1649, 44, 74, 2929, 4, 14, 131, 5, 35, 1649, 2, 5, 2929, 11, 1291, 28, 7146, 126, 1090, 2, 202, 111, 25934, 2, 3, 24582, 5, 31030, 2, 23, 64372, 2099, 3, 2244, 11, 7847, 16, 20, 1014, 10, 47810, 170, 3, 418, 426, 5, 1007, 5, 6, 1613, 4, 59, 33, 3, 7773, 53, 2, 29, 2244, 12, 1194, 5868, 4827, 47, 2929, 11, 1291, 4, 6, 37, 987, 2222, 7254, 131, 15, 4234, 86966, 12, 184, 1772, 4456, 183, 4, 29, 7529, 6905, 11, 105, 3, 541, 1450, 369, 36190, 2842, 2230, 7, 6912, 29, 6740, 2230, 59, 6, 1199, 3, 541, 313, 32, 916, 4, 5, 9047, 373, 12, 469, 50, 9, 231, 3, 2842, 2230, 2, 1738, 9, 14005,

## End